In [1]:
# !pip install -q openai

In [2]:
import openai
import time
import pandas as pd
import os
import numpy as np

ModuleNotFoundError: No module named 'openai'

In [468]:
df_final=pd.DataFrame()

In [469]:
openai.api_key='xxxxxx'

messages=[{"role":"system" , "content":"You are a kind helpful assistant"}, ]

In [470]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/')


df=pd.read_csv('enron_filtered_large.csv')
df=df[~(df['content']=='nan')]
df['content']=df['content'].astype(str)
df

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,message_id,subject,content
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...
1,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...
2,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...
3,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...
4,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...
...,...,...,...
481235,<13299843.1075840028954.JavaMail.evans@thyme>,RE: WSCC Tagging practices,I agree with Bob on #. I am indifferent on # ...
481236,<13266154.1075840029143.JavaMail.evans@thyme>,E-Tag Version 1.7 Implementation Delayed,TO: CONTROL AREAS TRANSMISSION PROVIDER...
481237,<13610289.1075840029369.JavaMail.evans@thyme>,RE: Common Scheduling Time Zone across WSCC,Good Morning Diana I just spoke with Duong a...
481238,<21704474.1075840029683.JavaMail.evans@thyme>,Late tags,Nothing is easy is it?? At the risk of the c...


In [471]:
df['Body']=df['subject']+df['content']
df

,message_id,subject,content,Body
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...,more about you A little bird told me about thi...
1,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...,LNG on the road trucking LNG to California - ...
2,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...,Amazon.com Password Assistance Greetings from ...
3,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...,Deferral Enrollment 2001 ...
4,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...,LNG Questions Russ: A couple of questions - C...
...,...,...,...,...
481235,<13299843.1075840028954.JavaMail.evans@thyme>,RE: WSCC Tagging practices,I agree with Bob on #. I am indifferent on # ...,RE: WSCC Tagging practices I agree with Bob on...
481236,<13266154.1075840029143.JavaMail.evans@thyme>,E-Tag Version 1.7 Implementation Delayed,TO: CONTROL AREAS TRANSMISSION PROVIDER...,E-Tag Version 1.7 Implementation Delayed TO: ...
481237,<13610289.1075840029369.JavaMail.evans@thyme>,RE: Common Scheduling Time Zone across WSCC,Good Morning Diana I just spoke with Duong a...,RE: Common Scheduling Time Zone across WSCC Go...
481238,<21704474.1075840029683.JavaMail.evans@thyme>,Late tags,Nothing is easy is it?? At the risk of the c...,Late tags Nothing is easy is it?? At the risk...


In [ ]:
df=df[df['Body'].str.split().str.len()<500]
df_enron=df

In [ ]:
df['result']=np.nan
df

In [ ]:
df=df.iloc[1054:4000]
df

In [ ]:
def gpt_out(df,x):
  message= """ Please categorise the following mail body into one of the 7 categories without any explaination, mention only the category as output: \n
    1) 'Company Business, Strategy, etc.'
    2) 'Purely Personal'
    3) 'Personal but in professional context'
    4) 'Logistic Arrangements(meeting scheduling, technical support, etc)'
    5) 'Employment arrangements(job seeking, hiring, recommendations, etc)'
    6) 'Document editing/checking(collaboration)'
    7) 'Payroll, finance, accounting etc.' """

  message=message + df['Body'].iloc[x]
  messages=[{'role':'user','content':message}]
  chat=openai.ChatCompletion.create(model='gpt-3.5-turbo',messages=messages)
  reply=chat.choices[0].message.content

  df['result'].iloc[x]=reply
  return df

In [ ]:
for x in range(len(df)):
  print(x)
  gpt_out(df,x)

In [460]:
df=df[~df['result'].isna()]
df

,message_id,subject,content,Body,result
1257,<11437093.1075857630021.JavaMail.evans@thyme>,Vandy Team - Get Together,Reminder - Reminder - Reminder Remember that...,Vandy Team - Get Together Reminder - Reminder ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
1258,<676855.1075857629878.JavaMail.evans@thyme>,Long term Gas Curve,John Hope youre well. We are doing some wor...,Long term Gas Curve John Hope youre well. We...,"1) 'Company Business, Strategy, etc.'"
1261,<21167674.1075857582725.JavaMail.evans@thyme>,Nigel Patterson,Can you swap me with Fletch. Try to make all...,Nigel Patterson Can you swap me with Fletch. ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
1262,<30685746.1075857583034.JavaMail.evans@thyme>,Video for Enron Management Conference,Please call her and schedule -----------------,Video for Enron Management Conference Please c...,"4) 'Logistic Arrangements(meeting scheduling, ..."
1266,<16819693.1075857624571.JavaMail.evans@thyme>,Recruiting Expenses,-----------------,Recruiting Expenses -----------------,"5) 'Employment arrangements(job seeking, hirin..."
...,...,...,...,...,...
1506,<21021838.1075855216316.JavaMail.evans@thyme>,RE: Enron open otc nymex swaps PGE.xls,try to net woth bank of america the cal,RE: Enron open otc nymex swaps PGE.xls try to ...,"4) Logistic Arrangements(meeting scheduling, t..."
1507,<18180299.1075852714205.JavaMail.evans@thyme>,RE:,crazy day....this day took a week off my life...,RE: crazy day....this day took a week off my l...,2) 'Purely Personal'
1508,<19576638.1075852714581.JavaMail.evans@thyme>,RE:,are you going to tell me or do i have to guess,RE: are you going to tell me or do i have to g...,"4) 'Logistic Arrangements(meeting scheduling, ..."
1509,<7885719.1075852713935.JavaMail.evans@thyme>,RE: specs,,RE: specs,"4) 'Logistic Arrangements(meeting scheduling, ..."


In [461]:
df_final=pd.read_csv('gpt3.5_output8.csv')

In [462]:
df_final=pd.concat([df_final,df])
df_final

,Unnamed: 0,message_id,subject,content,Body,result
0,0.0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...,more about you A little bird told me about thi...,"4) Logistic Arrangements (meeting scheduling, ..."
1,1.0,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...,LNG on the road trucking LNG to California - ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
2,2.0,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...,Amazon.com Password Assistance Greetings from ...,2) 'Purely Personal'
3,3.0,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...,Deferral Enrollment 2001 ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
4,4.0,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...,LNG Questions Russ: A couple of questions - C...,"4) 'Logistic Arrangements(meeting scheduling, ..."
...,...,...,...,...,...,...
1506,NaN,<21021838.1075855216316.JavaMail.evans@thyme>,RE: Enron open otc nymex swaps PGE.xls,try to net woth bank of america the cal,RE: Enron open otc nymex swaps PGE.xls try to ...,"4) Logistic Arrangements(meeting scheduling, t..."
1507,NaN,<18180299.1075852714205.JavaMail.evans@thyme>,RE:,crazy day....this day took a week off my life...,RE: crazy day....this day took a week off my l...,2) 'Purely Personal'
1508,NaN,<19576638.1075852714581.JavaMail.evans@thyme>,RE:,are you going to tell me or do i have to guess,RE: are you going to tell me or do i have to g...,"4) 'Logistic Arrangements(meeting scheduling, ..."
1509,NaN,<7885719.1075852713935.JavaMail.evans@thyme>,RE: specs,,RE: specs,"4) 'Logistic Arrangements(meeting scheduling, ..."


In [463]:
df_final.drop('Unnamed: 0',axis=1,inplace=True)
df_final

,message_id,subject,content,Body,result
0,<6184043.1075857585289.JavaMail.evans@thyme>,more about you,A little bird told me about this. Read it wh...,more about you A little bird told me about thi...,"4) Logistic Arrangements (meeting scheduling, ..."
1,<17858209.1075857584947.JavaMail.evans@thyme>,LNG on the road,trucking LNG to California - The short ans...,LNG on the road trucking LNG to California - ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
2,<14187877.1075857584924.JavaMail.evans@thyme>,Amazon.com Password Assistance,Greetings from Amazon.com. To finish resetti...,Amazon.com Password Assistance Greetings from ...,2) 'Purely Personal'
3,<31194989.1075857585233.JavaMail.evans@thyme>,Deferral Enrollment 2001,...,Deferral Enrollment 2001 ...,"4) 'Logistic Arrangements(meeting scheduling, ..."
4,<32084772.1075857584968.JavaMail.evans@thyme>,LNG Questions,Russ: A couple of questions - Check the DOE ...,LNG Questions Russ: A couple of questions - C...,"4) 'Logistic Arrangements(meeting scheduling, ..."
...,...,...,...,...,...
1506,<21021838.1075855216316.JavaMail.evans@thyme>,RE: Enron open otc nymex swaps PGE.xls,try to net woth bank of america the cal,RE: Enron open otc nymex swaps PGE.xls try to ...,"4) Logistic Arrangements(meeting scheduling, t..."
1507,<18180299.1075852714205.JavaMail.evans@thyme>,RE:,crazy day....this day took a week off my life...,RE: crazy day....this day took a week off my l...,2) 'Purely Personal'
1508,<19576638.1075852714581.JavaMail.evans@thyme>,RE:,are you going to tell me or do i have to guess,RE: are you going to tell me or do i have to g...,"4) 'Logistic Arrangements(meeting scheduling, ..."
1509,<7885719.1075852713935.JavaMail.evans@thyme>,RE: specs,,RE: specs,"4) 'Logistic Arrangements(meeting scheduling, ..."


In [464]:
# df['Body'].str.split().str.len()

In [465]:
from google.colab import drive
drive.mount('/content/drive')

os.chdir('/content/drive/My Drive/')

df_final.to_csv('gpt3.5_output8.csv')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [448]:
df_final.shape

(884, 5)

In [ ]:
# df_final['Body'].str.split().str.len().sum()

3786

In [ ]:
df_final[:61]

,Message-ID,Body,result
2524,<10955594.1075857581786.JavaMail.evans@thyme>,Sarah: Can you create a spreadsheet summarizin...,"4) Logistic Arrangements(meeting scheduling, t..."
2525,<21565966.1075851760611.JavaMail.evans@thyme>,IT Customer Satisfaction SurveyCustomer Suppor...,"1) Company Business, Strategy, etc."
2526,<5427197.1075845395123.JavaMail.evans@thyme>,"POSREP 10 MAYPOSREP HOEGH GALLEON, DATE: 10-Ma...","4) Logistic Arrangements(meeting scheduling, t..."
2528,<30087414.1075860456269.JavaMail.evans@thyme>,Re: Weekly Regulatory ReportFor the East (outs...,"1) Company Business, Strategy, etc."
2529,<24385849.1075849764540.JavaMail.evans@thyme>,i didnt know shirleys father died.,2) Purely Personal
...,...,...,...
3337,<7575753.1075856774195.JavaMail.evans@thyme>,REVISED: Restricted ListNeither ENA/RAC/EGF em...,"7) 'Payroll, finance, accounting etc.'"
4926,<26952446.1075858414924.JavaMail.evans@thyme>,Re: Contract Assignmenti think either one of u...,"Dear Leslie,\n\nThank you for reaching out. I’..."
4927,<9464774.1075845593381.JavaMail.evans@thyme>,ComEd PPA - Plano---------------------- Forwar...,"Hi Fred,\n\nThank you for forwarding the lates..."
4928,<10676610.1075842448434.JavaMail.evans@thyme>,"Risk Management PolicyDrew, I have asked Janet...","Hi Tony,\n\nThat sounds like a good plan. It's..."
